In [3]:
import sqlalchemy as db
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.decomposition import PCA
from collections import Counter

from mclLib.featurenames import *
from mclLib.server_info import *
from mclLib.scaler import *
from mclLib.modelpostfix import *

In [2]:
engine = create_engine(mysql_server_uri)
conn = engine.connect()

''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [5]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) 
split_filter = (br_full_data['dTradeTime'] < '2023-03-01')
br = br_full_data.loc[get_filter]
br_train = br.loc[split_filter]
br_test =  br.loc[~split_filter]


''' get features name'''
feature_names =  f_name_102

''' set X data '''
train_x = br_train[feature_names].to_numpy(dtype=np.float64)
test_x = br_test[feature_names].to_numpy(dtype=np.float64)

''' set X data '''

scale_method = STANDARD

model_name = 'test_svm_hyper_tune'

modelTester = ModelTester(engine, conn)
modelTester.setNpData(train_x)
modelTester.setScaler(scale_method)
modelTester.fitScale()
train_x = modelTester.np_data

modelTester.writeScalerToDB(
                    feature_names=feature_names,
                    model_name=model_name+onnx_,
                    )


put scale to  test_svm_hyper_tune.onnx  ends


In [6]:
# 임시테스트할때 
modelTester.setNpData(test_x)
modelTester.matchOldScaler(model_name + onnx_)
modelTester.fitScale()
test_x = modelTester.np_data

In [10]:

''' set y data '''
y_condition = (br_train['fMaxPowerAfterBuyWhile10'] < 0.026)
train_y = np.where(y_condition, 1, 0)

y_condition = (br_test['fMaxPowerAfterBuyWhile10'] < 0.026)
test_y = np.where(y_condition, 1, 0)

In [11]:
pca = PCA(n_components=20)
pca_model = pca.fit(train_x)
train_x_pca = pca_model.transform(train_x)
test_x_pca = pca_model.transform(test_x)

In [16]:

from sklearn import svm
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from ray import tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.lightgbm import TuneReportCheckpointCallback, TuneCallback
from sklearn.metrics import f1_score

import gc

def trainable(config, train_set, test_set):
    
    clf = None

    if config['kernel'] == 'poly':
        clf = svm.SVC(C=config['c'], gamma=config['gamma'], kernel=config['kernel'], degree=config['degree'])
    else:    
        clf = svm.SVC(C=config['c'], gamma=config['gamma'], kernel=config['kernel'])

    clf.fit(train_set[0], train_set[1])
    y_pred = clf.predict(test_set[0])
    return  {'score' : f1_score(test_set[1], y_pred)}


if __name__ == "__main__":

    


    config = {
        "c": tune.loguniform(0.1, 100.0),
        "gamma" : tune.loguniform(0.0001, 10.0),
        "kernel" : tune.choice(['linear', 'poly', 'rbf', 'sigmoid' ]),
        "degree": tune.randint(1, 5),
    }

    trainable = tune.with_parameters(trainable, train_set=(train_x_pca, train_y), test_set=(test_x_pca, test_y))
                                     

    tuner = tune.Tuner(
        trainable,
        tune_config=tune.TuneConfig(
            metric="score",
            mode="max",
            scheduler=ASHAScheduler(),
            num_samples=100,
        ),
        param_space=config,
        
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

2023-03-25 00:19:01,408	INFO worker.py:1553 -- Started a local Ray instance.


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
trainable_3883b_00005,2023-03-25_01-17-27,True,,cefd3312c65f40c780a9243cdf95c4bc,"5_c=5.9481,degree=2,gamma=2.4868,kernel=sigmoid",DESKTOP-EK3PPQN,1,127.0.0.1,19808,0.813442,3427.44,3427.44,3427.44,1679674647,0,,1,3883b_00005,0.0172863


2023-03-25 10:00:02,245	WARNING util.py:244 -- The `on_step_begin` operation took 0.594 s, which may be a performance bottleneck.


In [10]:
results.get_dataframe()

,binary_error,binary_logloss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,...,timesteps_since_restore,iterations_since_restore,warmup_time,config/boosting_type,config/learning_rate,config/metric,config/num_leaves,config/objective,config/verbose,logdir
0,0.143268,0.364938,1.218307,True,NaN,NaN,64,3c8f5_00000,cd3e997262d842e9b369dc25f77cf2ed,2023-03-18_21-59-34,...,0,64,0.010888,gbdt,1.126300e-02,"[binary_error, binary_logloss]",516,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
1,0.143576,0.361500,0.378026,True,NaN,NaN,100,3c8f5_00001,ab0bd842d2d54ddeae9d9a705ce8df47,2023-03-18_21-59-12,...,0,100,0.009748,dart,3.881620e-02,"[binary_error, binary_logloss]",60,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
2,0.142989,0.409209,1.940951,True,NaN,NaN,100,3c8f5_00002,86c97d48754f473f9711a2095ef92e42,2023-03-18_22-01-21,...,0,100,0.007096,gbdt,4.166556e-04,"[binary_error, binary_logloss]",986,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
3,0.142989,0.477892,0.843387,True,NaN,NaN,100,3c8f5_00003,ae859eb18b374b288e073200259fef00,2023-03-18_22-00-42,...,0,100,0.010087,dart,5.618311e-07,"[binary_error, binary_logloss]",483,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
4,0.142989,0.413072,1.401657,True,NaN,NaN,100,3c8f5_00004,6ab26cd642564622a0ae990a924ccbf5,2023-03-18_22-00-58,...,0,100,0.012063,gbdt,1.095447e-04,"[binary_error, binary_logloss]",834,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
5,0.142989,0.477896,1.666445,True,NaN,NaN,100,3c8f5_00005,95ee4e2387a140e4922bab6917eae74a,2023-03-18_22-01-59,...,0,100,0.010206,dart,4.514985e-08,"[binary_error, binary_logloss]",931,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
6,0.143312,0.361418,1.074867,True,NaN,NaN,16,3c8f5_00006,d131bae84ad4408b9dfe348c7e99d2c8,2023-03-18_21-58-35,...,0,16,0.010683,gbdt,5.162122e-02,"[binary_error, binary_logloss]",481,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
7,0.142989,0.477896,0.700241,True,NaN,NaN,100,3c8f5_00007,93c7b4c157774e2895818e3de15086fd,2023-03-18_22-00-16,...,0,100,0.008485,dart,1.868142e-08,"[binary_error, binary_logloss]",295,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
8,0.142989,0.414096,0.727958,True,NaN,NaN,100,3c8f5_00008,d131bae84ad4408b9dfe348c7e99d2c8,2023-03-18_22-00-02,...,0,100,0.010683,gbdt,3.605725e-05,"[binary_error, binary_logloss]",248,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
9,0.142989,0.477896,1.228474,True,NaN,NaN,100,3c8f5_00009,ab0bd842d2d54ddeae9d9a705ce8df47,2023-03-18_22-01-38,...,0,100,0.009748,dart,2.228663e-08,"[binary_error, binary_logloss]",364,binary,-1,C:\Users\MJ\ray_results\trainable_2023-03-18_2...
